In [1]:
import theano
import theano.tensor as T
import lasagne
import numpy as np
import sklearn.datasets
from __future__ import print_function
import os
import warnings
warnings.filterwarnings('ignore', module='lasagne')

/home/pauey/spcs-ai/lib/python3.4/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [10]:
from keras.datasets import mnist
import matplotlib.pyplot as plt
# load (downloaded if needed) the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


TypeError: 'DataSet' object is not iterable

In [ ]:
input_shape = dataset['train']['X'][0].shape
l_in = lasagne.layers.InputLayer(
shape=(None, input_shape[0], input_shape[1], input_shape[2]))

l_conv1 = lasagne.layers.Conv2DLayer(
l_in,
    
num_filters=32, filter_size=(5, 5),
    
nonlinearity=lasagne.nonlinearities.rectify,
    
W=lasagne.init.HeNormal(gain='relu'))

l_pool1 = lasagne.layers.MaxPool2DLayer(l_conv1, pool_size=(2, 2))

l_conv2 = lasagne.layers.Conv2DLayer(
l_pool1, num_filters=32, filter_size=(5, 5),
nonlinearity=lasagne.nonlinearities.rectify,
W=lasagne.init.HeNormal(gain='relu'))
l_pool2 = lasagne.layers.MaxPool2DLayer(l_conv2, pool_size=(2, 2))

l_hidden1 = lasagne.layers.DenseLayer(
l_pool2, num_units=256,
nonlinearity=lasagne.nonlinearities.rectify,
W=lasagne.init.HeNormal(gain='relu'))


l_hidden1_dropout = lasagne.layers.DropoutLayer(l_hidden1, p=0.5)

l_output = lasagne.layers.DenseLayer(
l_hidden1_dropout,
    
num_units=10,
nonlinearity=lasagne.nonlinearities.softmax)



In [ ]:
true_output = T.ivector('true_output')
objective = lasagne.objectives.Objective(l_output,
loss_function=lasagne.objectives.categorical_crossentropy)

loss_train = objective.get_loss(target=true_output, deterministic=False)
loss_eval = objective.get_loss(target=true_output, deterministic=True)
all_params = lasagne.layers.get_all_params(l_output)

updates = lasagne.updates.adadelta(loss_train, all_params)
train = theano.function([l_in.input_var, true_output], loss_train, updates=updates)

get_output = theano.function([l_in.input_var], l_output.get_output(deterministic=True))

In [ ]:
BATCH_SIZE = 100
N_EPOCHS = 10

batch_idx = 0

epoch = 0
while epoch < N_EPOCHS:
    
train(dataset['train']['X'][batch_idx:batch_idx + BATCH_SIZE],
dataset['train']['y'][batch_idx:batch_idx + BATCH_SIZE])
batch_idx += BATCH_SIZE

if batch_idx >= dataset['train']['X'].shape[0]:
    
batch_idx = 0

epoch += 1

val_output = get_output(dataset['valid']['X'])

val_predictions = np.argmax(val_output, axis=1)

accuracy = np.mean(val_predictions == dataset['valid']['y'])
print("Epoch {} validation accuracy: {}".format(epoch, accuracy))